In [ ]:
!pip -q install "transformers>=4.42" "datasets>=2.20" "scikit-learn>=1.5"

In [ ]:
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments
)
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support
from datasets import Dataset
import pandas as pd
import numpy as np
import torch


In [ ]:
test_df = pd.read_csv("src/data/processed_bert/test_perturbation.csv")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(labels, predictions)
    macro_f1 = f1_score(labels, predictions, average='macro')
    weighted_f1 = f1_score(labels, predictions, average='weighted')

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions,
        average=None,
        labels=[0, 1, 2]
    )

    return {
        'accuracy': accuracy,
        'f1_macro': macro_f1,
        'f1_weighted': weighted_f1,
        'hate_speech_precision': precision[0],
        'hate_speech_recall': recall[0],
        'hate_speech_f1': f1[0],
        'offensive_precision': precision[1],
        'offensive_recall': recall[1],
        'offensive_f1': f1[1],
        'neither_precision': precision[2],
        'neither_recall': recall[2],
        'neither_f1': f1[2],
    }

In [ ]:
def tokenize_dataset(df, tokenizer, desc="original train"):
    dataset = Dataset.from_pandas(df[['clean_text', 'label']])

    def tokenize_batch(examples):
        return tokenizer(
            examples['clean_text'],
            padding='max_length',
            truncation=True,
            max_length=128
        )

    tokenized = dataset.map(tokenize_batch, batched=True, desc=f"Tokenizing {desc}")
    tokenized = tokenized.rename_column('label', 'labels')
    tokenized = tokenized.remove_columns(['clean_text'])
    tokenized.set_format('torch')

    return tokenized

In [ ]:
def eval(model_name):
  model_path = f"/content/{model_name}/best_model"
  output_dir = f"/content/{model_name}/output"
  model = BertForSequenceClassification.from_pretrained(model_path)
  tokenizer = BertTokenizer.from_pretrained(model_path)

  args = TrainingArguments(
      output_dir=output_dir,
      per_device_eval_batch_size=32,
      report_to='none',
  )

  trainer = Trainer(
      model=model,
      tokenizer=tokenizer,
      compute_metrics=compute_metrics,
      args=args
  )

  test_tok = tokenize_dataset(test_df, tokenizer, desc="test")

  print("Generating test predictions...")
  test_output = trainer.predict(test_tok)
  test_preds = np.argmax(test_output.predictions, axis=1)
  test_probs = torch.softmax(torch.tensor(test_output.predictions), dim=-1).numpy()

  test_predictions_df = pd.DataFrame({
        'true_label': test_df['label'].values,
        'predicted_label': test_preds,
        'prob_hate': test_probs[:, 0],
        'prob_offensive': test_probs[:, 1],
        'prob_neither': test_probs[:, 2]
    })

  test_predictions_df.to_csv(f"{output_dir}/test_predictions.csv", index=False)

  metrics_df = pd.DataFrame([test_output.metrics])
  metrics_df.to_csv(f"{output_dir}/metrics.csv", index=False)

  print(f"  Saved to {output_dir}/")
  return test_output.metrics


In [ ]:
metrics_orig = eval('bert_original')
metrics_aug = eval('bert_augmented')